## Evaluating Trained Models on the [BabyLM Eval Pipeline](https://github.com/babylm/evaluation-pipeline-2023)

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'GPT-9.0M-2L-4H-512C-1024I'
checkpoint = 'checkpoint-6624'

tokenizer = AutoTokenizer.from_pretrained('10k-tok')
model = AutoModelForCausalLM.from_pretrained(f'results/models/{model_name}/{checkpoint}')

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: Incorrect path_or_model_id: 'results/models/GPT-9.0M-2L-4H-512C-1024I/checkpoint-6624'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

##### Setup
1. Create a new environment for the pipeline, as there are a lot of external dependencies that may interfere with your development environment. Follow installation instructions at [BabyLM/evaluation-pipeline](https://github.com/babylm/evaluation-pipeline). 
    - If their torch version does not match your CUDA drivers, remove the version specification in `setup.py`. 
    - They use `python==3.10.12` in their example notebook; this does not work for my CUDA installation.
        Specifically, there is a [bug](https://discuss.pytorch.org/t/issues-on-using-nn-dataparallel-with-python-3-10-and-pytorch-1-11/146745/13) requiring you to change `lib/python3.10/site-packages/torch/cuda/nccl.py`, at line `51`: from `collections` to `collections.abc`. 

2. Unzip the dataset: `unzip filter_data.zip`. 


```sh
# create env 
conda create -n babylm python
cd evaluation-pipeline # you need to make sure you've fetched this submodule with git
pip install -e '.[dev]'
pip install torch 

unzip filter_data.zip
```

##### Running
I explain how to run the pipeline below; but strongly recommend writing/converting to a `.py` script instead if you are running a batch of these. Notebooks can time out after a while, or you know, you might want to close your computer and touch some grass while this stuff is running. See [delftblue-guide.md](./delftblue-guide.md) for more info. 

Make sure you have the tokenizer files (or symlink to it) in your model's dir. 

```sh 
# symlink the tokenizer into the model's directory
ln -s 10k-tok results/models/{model_name}

python babylm_eval.py 'path/to/model_and_tokenizer' decoder # encoder for BERT
./finetune_all_tasks.sh 'path/to/model_and_tokenizer' 
./collect_results.py 'path/to/model_and_tokenizer'
``` 

#### Push to HuggingFace


In [ ]:
model.push_to_hub(model_name)